In [1]:
#!pip install rdflib
#!pip install pydot2
#!pip install pydotplus

In [2]:
import rdflib
from rdflib.namespace import RDFS
from rdflib import URIRef, BNode, Literal
import re
from collections import defaultdict
import numpy as np
import pandas as pd
import networkx as nx
from networkx.drawing.nx_pydot import write_dot
import matplotlib.pyplot as plt

/Users/bbalaji/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [3]:
def ns(url):
  url = url.replace("http://buildsys.org/ontologies/brick#", "")
  url = url.replace("http://buildsys.org/ontologies/brickFrame#", "")
  return url

BRICKF = rdflib.Namespace('http://buildsys.org/ontologies/brickFrame#')
BRICK  = rdflib.Namespace('http://buildsys.org/ontologies/brick#')
TAG    = rdflib.Namespace('http://buildsys.org/ontologies/brickTag#')
TAGSET = rdflib.Namespace('http://buildsys.org/ontologies/brickTagSet#')

g = rdflib.Graph()
g.bind('bf', BRICKF)
g.bind('tag', TAG)
g.bind('ts', TAGSET)
g.bind(':', BRICK)
result = g.parse('BrickFrame.ttl', format='n3')
print(len(g))

130


### Load Tag and TagSets from Definition

In [4]:
dfTags=pd.read_excel('Schema Engineering.xlsx',"Tags")
schemaTags=set(pd.unique(dfTags.Tag.dropna().ravel()))
len(schemaTags)
dfTags.head()

,Dimension,Tag,Definition,Related Terminologies,hasSynonym,Author,Reviewed,Review Count,Comments
0,Equipment>HVAC,HX,Heat Exchanger,NaN,NaN,NaN,NaN,NaN,NaN
1,Equipment>Fire Safety System,FCP,Fire Control Panel,NaN,NaN,NaN,NaN,NaN,NaN
2,Equipment>Water System,CWS,Chilled Water System,NaN,NaN,NaN,NaN,NaN,NaN
3,Equipment>Water System,DHWS,Domestic Hot Water System,NaN,NaN,NaN,NaN,NaN,NaN
4,Equipment>Water System,HWS,Hot Water System,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
dfTagSets=pd.read_excel('Schema Engineering.xlsx',"TagSets")
schemaTagSets=set()
for ts in pd.unique(dfTagSets.TagSet.dropna().ravel()):
  schemaTagSets.add(ts.replace(' ','_'))
for ts in pd.unique(dfTagSets.hasSynonym.dropna().ravel()):
  for ts2 in ts.split(","):
   schemaTagSets.add(ts2.replace(' ','_'))
len(schemaTagSets)
dfTagSets.head()

,Dimension,TagSet,isPointOf,isPartOf,hasSynonym,Definition,hasUnit,hasAbbreviation,can be named? (can be instantiated?),can be unnamed? (cannot be instantiated?),Author,Reviewed,Review Count,Discussion
0,Equipment,Fire Safety System,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Equipment,Solar Panel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Equipment,Lighting System,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Equipment,Weather,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Equipment,Energy Storage,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
schemaUsedTags=set()
schemaTagSetTags={}
for ts in schemaTagSets:
  schemaUsedTags.update(ts.split('_'))
  schemaTagSetTags[ts]=set(ts.split('_'))
schemaMissingTags=schemaUsedTags - schemaTags
print("Missing Tags:" + str(len(schemaMissingTags)))
print(schemaMissingTags)

Missing Tags:32
set([u'', u'Dew', u'Ratio', u'Integrative', u'Elevator', u'Electrical', u'Condensor', u'Factor', u'Override', u'Cutout', u'PreHeat', u'Failure', u'Curtailment', u'Operating', u'Gain', u'Protect', u'Tank', u'Isolation', u'Timer', u'Server', u'Chiller', u'Ice', u'Inverter', u'Fixed', u'Leaving', u'Vent', u'Point', u'Duct', u'Weather', u'Entering', u'Close', u'Thumbwheel'])


In [7]:
def IndivName(name):
  return re.sub(r'\s','_',re.sub(r'[^\d\w\s]', '', name))

### Write Tags

write header

In [8]:
nsTag   = ":"
fo = open('BrickTag.ttl', 'w')
fo.write("""@prefix dc: <http://purl.org/dc/elements/1.1/> .
@prefix DUL: <http://www.loa-cnr.it/ontologies/DUL.owl#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix ssn: <http://purl.oclc.org/NET/ssnx/ssn#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .
@prefix time: <http://www.w3.org/2006/time#> .
@prefix unit: <http://qudt.org/vocab/unit#> .
@prefix gbXML: <http://www.gbxml.org/schema#> .
@prefix haystack: <http://project-haystack.org/tag/> .\n""")
fo.write("@prefix bf:  <http://buildsys.org/ontologies/BrickFrame#> .\n")
fo.write("@prefix : <http://buildsys.org/ontologies/BrickTag#> .\n\n")
fo.write("<http://buildsys.org/ontologies/BrickTag>  a owl:Ontology ;\n")
fo.write("\towl:imports <http://buildsys.org/ontologies/BrickFrame> ;\n")
fo.write('\trdfs:comment "Domain Tag Definition"@en .\n\n')

Write Tag Hierachy

In [9]:
brickTags={}
for hir in pd.unique(dfTags.Dimension.dropna().ravel()):
  tags=hir.split('>')
  atags=""
  for i in range(len(tags)):
    tag=tags[i]
    otags=atags
    atags=(atags+"_"+tag).strip("_")
    if atags not in brickTags:
      indivLocName=nsTag+IndivName(atags)
      if i>0:
        fo.write("\n "+indivLocName+"  rdfs:subClassOf   "+brickTags[otags]+";")
      else:
        fo.write("\n "+indivLocName+"  rdfs:subClassOf   bf:Tag;")
      fo.write('\n\t\t\t bf:isHierarchical  "";')
      fo.write('\n\t\t\t rdfs:label "'+tag+'"@en .\n')
      brickTags[atags]=indivLocName;
      parent=tag;

Add tag leaves

In [10]:
# create location individuals
for idx in dfTags.index:
  #parent=brickTags[str(dfTags.loc[idx, "Dimension"]).split('>')[-1]]
  parent=brickTags[str(dfTags.loc[idx, "Dimension"]).replace('>','_')]
  indivLocName=nsTag + IndivName(str(dfTags.loc[idx, "Tag"]))
  fo.write("\n "+indivLocName+"  rdfs:subClassOf   "+parent+";")
  fo.write('\n\t\t\t rdfs:label "'+str(dfTags.loc[idx, "Tag"])+'"@en .\n')
  brickTags[tag]=indivLocName;
  #print(parent,indivLocName)

In [11]:
fo.close()

### Write TagSets

write headers

In [12]:
nsTag= "tag:" # "ts:"
nsTagSet= ":" # "ts:"
fo = open('Brick.ttl', 'w')
fo.write("""@prefix dc: <http://purl.org/dc/elements/1.1/> .
@prefix DUL: <http://www.loa-cnr.it/ontologies/DUL.owl#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix ssn: <http://purl.oclc.org/NET/ssnx/ssn#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .
@prefix time: <http://www.w3.org/2006/time#> .
@prefix unit: <http://qudt.org/vocab/unit#> .
@prefix gbXML: <http://www.gbxml.org/schema#> .
@prefix haystack: <http://project-haystack.org/tag/> .\n""")
fo.write("@prefix bf:  <http://buildsys.org/ontologies/BrickFrame#> .\n")
fo.write("@prefix tag: <http://buildsys.org/ontologies/BrickTag#> .\n")
fo.write("@prefix :    <http://buildsys.org/ontologies/Brick#> .\n\n")
fo.write("<http://buildsys.org/ontologies/Brick>  a owl:Ontology ;\n")
fo.write("\towl:imports <http://buildsys.org/ontologies/BrickFrame> ;\n")
fo.write("\towl:imports <http://buildsys.org/ontologies/BrickTag> ;\n")
fo.write('\trdfs:comment "Domain TagSet Definition"@en .\n\n')

Write TagSet Hierachy

In [13]:
brickTagSets={}
for hir in pd.unique(dfTagSets.Dimension.dropna().ravel()):
  tags=hir.split('>')
  atags=""
  for i in range(len(tags)):
    tag=tags[i]
    otags=atags
    #Joern's definition of atags (i.e. class names)
    #atags=(atags+"_"+tag).strip('_')
    #Bharath's definition of class names (atags)
    atags=tag.strip('_')
    if atags not in brickTagSets:
      indivLocName=nsTagSet + IndivName(atags)
      if i>0:
        fo.write("\n "+indivLocName+"  rdfs:subClassOf   "+brickTagSets[otags]+";")
      else:
        fo.write("\n "+indivLocName+"  rdfs:subClassOf   bf:TagSet;")
      #fo.write('\n\t\t\t bf:isHierarchical  "";')
      fo.write('\n\t\t\t rdfs:label "'+tag+'"@en .\n')
      brickTagSets[atags]=indivLocName;

Add TagSets leaves

In [14]:
# create location individuals
for idx in dfTagSets.index:
  tagsets=set([str(dfTagSets.loc[idx, "TagSet"])]) | set(str(dfTagSets.loc[idx, "hasSynonym"]).split(","))
  ots=None
  for tagset in tagsets:
    if tagset!="nan":
        #tagset=str(dfTagSets.loc[idx, "TagSet"])
      #Bharath's definition of parent
      parent= brickTagSets[str(dfTagSets.loc[idx, "Dimension"]).split('>')[-1]]
      #Joern's definition of parent
      #parent= brickTagSets[str(dfTagSets.loc[idx, "Dimension"]).replace('>',"_")]
      indivLocName=nsTagSet + IndivName(tagset)
      fo.write("\n "+indivLocName+"  rdfs:subClassOf   "+parent+";")
      for tag in tagset.split():
         #fo.write('\n\t\t\t bf:hasTag  tag:'+tag+';')
         fo.write('\n\t\t\t rdfs:subClassOf [ a owl:Restriction ; owl:onProperty :hasTag ; owl:someValuesFrom tag:'+tag+' ];')
      if ots:
        fo.write('\n\t\t\t owl:equivalentClass '+ots+' ;')
      fo.write('\n\t\t\t rdfs:label "'+str(dfTagSets.loc[idx, "TagSet"])+'"@en .\n')
      brickTagSets[tagset]=indivLocName;
      ots=indivLocName;
      #print(parent,indivLocName)

KeyError: 'nan'

In [15]:
fo.close()

### New Structure

Classify Tags by Type

In [ ]:
#Classify tags by dimension from "Tags" sheet in Schema Engineering
dfMeasTags=dfTags.loc[dfTags.Dimension == "MeasurementProperty>PhysicalProperties"]
dfLocTags=dfTags.loc[dfTags.Dimension == "Location"]
dfPointTags=dfTags.loc[dfTags.Dimension == "Point"]
dfEquipTags=dfTags.loc[dfTags.Dimension.str.startswith("Equipment")]

#Identify dimensions based on tags
def getPointDim(istr):
  for pt in dfPointTags.Tag:
    if pt in str(istr): return pt
  return "UndefinedPoint"

def getMeasDim(istr):
  for pt in dfMeasTags.Tag:
    if pt in str(istr): return pt
  return "UndefinedMeasurement"

def getLocDim(istr):
  for pt in dfLocTags.Tag:
    if pt in str(istr): return pt
  return "UndefinedLocation"

def getEquipDim(istr):
  for pt in dfEquipTags.Tag:
    if pt in str(istr): return pt
  return "UndefinedEquipment"

def getLastDim(istr):
  if ">"   not in istr:
    return "UndefinedEquipment"
  else:
    sstr=istr.split(">")
    return IndivName(sstr[-1])  

In [ ]:
#Classify each tagset as a dimension based on their tags
dfTagSets['PointDim']=dfTagSets.TagSet.apply(getPointDim)
dfTagSets['MeasurementDim']=dfTagSets.TagSet.apply(getMeasDim)
dfTagSets['LocationDim']=dfTagSets.TagSet.apply(getLocDim)
#dfTagSets['EquipmentDim']=dfTagSets.TagSet.apply(getEquipDim)
dfTagSets['EquipmentDim']=dfTagSets.Dimension.apply(getLastDim)
dfPointTagSets=dfTagSets.loc[dfTagSets.Dimension.str.startswith("Point")]

In [ ]:
#Dictionary mapping tagset to set of its constituent tags
tagsets={}
#Dictionary mapping tagset to its list of parents (i.e. immediate superclass)
tagsetsSup={}
#Synonyms: List of tagset sets
equivTagSets=[]
for dim in ['PointDim', 'MeasurementDim', 'LocationDim', 'EquipmentDim']:
  dimT=dim.replace('Dim', '')
  #Basic dimensions have TagSet as superclass
  tagsetsSup[dimT]=['TagSet']
  #Child tagsets are stored in tagsets
  tagsets[dimT]=set()
  for dimL in pd.unique(dfPointTagSets[dim].dropna().ravel()):
    #Every Tagset dimension has Point, Location, Measurement and Equip as their superclass
    #Note: This is a flat hierarchy
    tagsetsSup[dimL]=[dimT]
    #if not dimL.startswith("Undefined"):
    #Add to dictionary mapping tagsets to list of tags
    tagsets[dimL]=set(dimL.split('_'))

#Working with tagset leaves    
for idx in dfPointTagSets.index:
  #List of all tagsets for this point, including synonyms
  tss=set([str(dfTagSets.loc[idx, "TagSet"])]) | set(str(dfTagSets.loc[idx, "hasSynonym"]).split(","))
  tss2=set()
  #remove blanks
  if "nan" in tss: tss.remove("nan")
    
  for tagset in tss:
    if tagset!="nan":
      #replace space with '_'. Keep only alpha numeric characters.
      tagset=IndivName(tagset)
      #Compile synonyms in a set
      tss2.add(tagset)
      #Compile dictionary mapping tagset to list of tags
      tagsets[tagset] = set(tagset.split('_'))
      #Get the equipment dimension to which the point is associated with
      equip=dfTagSets.loc[idx, 'EquipmentDim']
      #Add the valid equipment dimension tags to tagsets dictionary
      if equip!="nan" and equip != "UndefinedEquipment" and equip not in tagsets[tagset]:
        for tag in equip.split('_'):
          tagsets[tagset].add(tag)
        
      #define parents and add it to tagsetsSup dictionary
      parents=[]
      for dim in ['PointDim', 'MeasurementDim', 'LocationDim', 'EquipmentDim']:
        parents.append(dfTagSets.loc[idx, dim])
      tagsetsSup[tagset]=parents
  #Compile synonyms set list.
  if len(tss2)>1:
    equivTagSets.append(tss2)

In [ ]:
#Convert tagsetsSup dictionary of lists to dictionary of sets
tagsetsSupAll={}
for tsA in tagsetsSup:
  if tsA not in tagsetsSupAll: tagsetsSupAll[tsA]=set()
  for tsB in tagsetsSup[tsA]:
    tagsetsSupAll[tsA].add(tsB)
#Count all tagsets parents
c=0
for tsA in tagsetsSupAll:
  for tsB in tagsetsSupAll[tsA]:
    c+=1
c


In [ ]:
i=0
#Dictionary mapping missing tagsets to its set of parents
missingRel={}
for tsA in tagsets:
  i=i+1
  j=0
  for tsB in tagsets:
    j=j+1
    if tsA==tsB: #j <= i or 
      pass
    #Using set relationship to determine parent child relationship in the hierarchy
    #If set of tags in tsB is a subset of set of tags in tsA, then tsB is a parent of tsA
    if tagsets[tsA] > tagsets[tsB]: # tsB parentOf tsA
      if tsA not in tagsetsSup:
        #If tsA is not already in the tagsetsSup dictionary, add it as a missing term in missingRel dictionary
        if tsA not in missingRel: missingRel[tsA]=set()
        #Also add the relevant parent information to missingRel
        missingRel[tsA].add(tsB)  
      else:
        #Ensure that all the superclasses of tsA are subClassOf tsB
        anySup=False
        for tsC in tagsetsSup[tsA]:
          if tsC in tagsets and tagsets[tsC] > tagsets[tsB]: # tsB parentOf tsC
            anySup = True
        #If there is a tsB which is not a superclass of one of the superclass of tsA, add it in missingRel dictionary
        if not anySup:
          if tsA not in missingRel: missingRel[tsA]=set()
          missingRel[tsA].add(tsB)
#Count list of tagsets parents in missingRel dictionary
c=0
for tsA in missingRel:
  for tsB in missingRel[tsA]:
    c+=1
c

In [ ]:
# remove redundant entries
ignoreTagSets=['Point','Measurement','Location','Equipment',"UndefinedPoint",
               "UndefinedEquipment","UndefinedLocation","UndefinedMeasurement"]
remTsA=set(ignoreTagSets)
for tsA in missingRel:
  remTsB=set(ignoreTagSets)
  for tsB in missingRel[tsA]:
    for tsC in missingRel[tsA]:
      #If tsC is superclass of tsA, tsB is superclass of tsA and tsB is superclass of tsC, then tsB is redundant
      if tagsets[tsC] > tagsets[tsB]:
        remTsB.add(tsB)
  #remove redundant superclass entries
  missingRel[tsA]=missingRel[tsA].difference(remTsB)
  #If there are no missing superclasses for tsA, add it to remove set
  if len(missingRel[tsA])==0: remTsA.add(tsA)
#Remove redundant missingRel entries
for tsA in remTsA:
  if tsA in missingRel:
    del missingRel[tsA]

#Add all the remaining missingRel terms to tagsetsSupAll dictionary
for tsA in missingRel:
  if tsA not in tagsetsSupAll: tagsetsSupAll[tsA]=set()
  for tsB in missingRel[tsA]:
    tagsetsSupAll[tsA].add(tsB)

#Check count of final missingRel terms
c=0
for tsA in missingRel:
  for tsB in missingRel[tsA]:
    c+=1
c

In [ ]:
def findMin(S, R):
  maxCost = 0
  minElement = None
  for i in S:
    try:
      cost = len(S[i].intersection(R))
      if cost > maxCost:
        maxCost = cost
        minElement = i
    except:
      # Division by zero, ignore
      pass
  return minElement


In [ ]:
# compute minimal set cover of parents
redundantSup={}
ignoreTagSets=['Point','Measurement','Location','Equipment',"UndefinedPoint","UndefinedEquipment",
               "UndefinedLocation","UndefinedMeasurement"]
remTsA=set(ignoreTagSets)
tagsetsSupMin={}
for tsA in tagsetsSupAll:
  if tsA not in tagsets:
    tagsetsSupMin=tagsetsSupAll[tsA]
    print(tsA+" not in tagsets")
    continue
  unassigned=set(tagsets[tsA])
  S={}
  minCov=set()
  for tsB in tagsetsSupAll[tsA]: 
    if tsB!=tsA and tsB in tagsets and len(tagsets[tsB])>0:
      S[tsB]=set(tagsets[tsB])
  while len(unassigned)>0 and len(S)>0:
    tsB=findMin(S, unassigned)
    if tsB is None: break
    unassigned=unassigned.difference(S[tsB])
    minCov.add(tsB)
    del S[tsB]
  for tag in unassigned:
    if tag!=tsA:
      minCov.add(tag)
  if minCov:
    tagsetsSupMin[tsA]=minCov
    #print(tsA+": "+str(minCov))
#Check count
c=0
for tsA in tagsetsSupMin:
  for tsB in tagsetsSupMin[tsA]:
    c+=1
c

In [ ]:
# identify redundant parents exceeding minimal set cover
redundantSup={}
for tsA in tagsetsSupAll:
  if tsA in tagsetsSupMin:
    redundantSup[tsA]=tagsetsSupAll[tsA].difference(tagsetsSupMin[tsA])
#Check count
c=0
for tsA in redundantSup:
  for tsB in redundantSup[tsA]:
    c+=1
c

In [ ]:
for tsA in redundantSup:
  for tsB in redundantSup[tsA]:
    if tsA in tagsetsSupAll and tsB in tagsetsSupAll[tsA]:
      if not tsB.startswith("Undefined") and tsB!="":  
        tagsetsSupAll[tsA].remove(tsB)
    #g.remove((tagsetsNode[tsA],URIRef('http://www.w3.org/2000/01/rdf-schema#subClassOf'),tagsetsNode[tsB]))
for tsA in tagsetsSupMin:
  for tsB in tagsetsSupMin[tsA]:
    pass
    #if tsA not in tagsetsSupAll: tagsetsSupAll[tsA]=set([tsB])
    #else: tagsetsSupAll[tsA].add(tsB)
    #g.add((tagsetsNode[tsA],URIRef('http://www.w3.org/2000/01/rdf-schema#subClassOf'),tagsetsNode[tsB]))
#Check count
c=0
for tsA in tagsetsSupAll:
  for tsB in tagsetsSupAll[tsA]:
    c+=1
c

In [ ]:
c=0
for tsA in tagsets: c+=len(tagsets[tsA])
c

In [ ]:
for tsA in tagsetsSupAll:
  if tsA in tagsets:
    uniqueTags=set(tagsets[tsA])
    if tsA in tagsetsSupAll:
      for tsB in tagsetsSupAll[tsA]:
        if tsB in tagsets:
          commontags=uniqueTags.intersection(tagsets[tsB])
          for tag in commontags:
            if tag in tagsets[tsA]:
              #g.remove((tagsetsNode[tsA],None,tagsetsTagsNode[tsA][tag]))
              tagsets[tsA].remove(tag)
c=0
for tsA in tagsets: c+=len(tagsets[tsA])
c

In [ ]:
nsTag= "tag:" # "ts:"
nsTagSet= ":" # "ts:"
fo = open('BrickV2.ttl', 'w')
fo.write("""@prefix dc: <http://purl.org/dc/elements/1.1/> .
@prefix DUL: <http://www.loa-cnr.it/ontologies/DUL.owl#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix ssn: <http://purl.oclc.org/NET/ssnx/ssn#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .
@prefix time: <http://www.w3.org/2006/time#> .
@prefix unit: <http://qudt.org/vocab/unit#> .
@prefix gbXML: <http://www.gbxml.org/schema#> .
@prefix haystack: <http://project-haystack.org/tag/> .\n""")
fo.write("@prefix bf:  <http://buildsys.org/ontologies/BrickFrame#> .\n")
fo.write("@prefix tag: <http://buildsys.org/ontologies/BrickTag#> .\n")
fo.write("@prefix :    <http://buildsys.org/ontologies/Brick#> .\n\n")
fo.write("<http://buildsys.org/ontologies/Brick>  a owl:Ontology ;\n")
fo.write("\towl:imports <http://buildsys.org/ontologies/BrickFrame> ;\n")
fo.write("\towl:imports <http://buildsys.org/ontologies/BrickTag> ;\n")
fo.write('\trdfs:comment "Domain TagSet Definition"@en .\n\n')

In [ ]:
brickTagSets={}
for dim in ['PointDim','MeasurementDim','LocationDim','EquipmentDim']:
  #Write all the high level dimensions
  dimT=dim.replace('Dim','')
  fo.write("\n :"+dimT+"  rdfs:subClassOf   bf:TagSet;")
  fo.write('\n\t\t\t bf:isHierarchical  "";')
  #fo.write('\n\t\t\t rdfs:label "'+dimT+' Dimension"@en .\n')
  #Bharath: Removing "Dimension" from labels
  fo.write('\n\t\t\t rdfs:label "'+dimT+'"@en .\n')
  #Write the dimensions defined in dfPointTagSets. Note: this is a flat hierarchy
  for dimL in pd.unique(dfPointTagSets[dim].dropna().ravel()):
    fo.write("\n :"+dimL+"  rdfs:subClassOf   :"+dimT+";")
    #Bharath: Removing restriction of abstract classes
    #fo.write('\n\t\t\t bf:isHierarchical  "";')
    fo.write('\n\t\t\t rdfs:label "'+dimL+'"@en .\n')
    brickTagSets[dimL]=dimL;

In [ ]:
#Write the tagset leaves
for ts in tagsets:
  supClass=""
  if ts in tagsetsSupAll:
    for ts2 in tagsetsSupAll[ts]:
      if ts2=="TagSet": supClass+=", bf:"+ts2;
      else: supClass+=", :"+ts2;
  if supClass=="":
    supClass=":Equipment"
  #Remove extraneous commas and spaces to write the final list of superclasses for tagset
  fo.write("\n :"+ts+"  rdfs:subClassOf   "+supClass.strip(',').strip()+";")
  #fo.write('\n\t\t\t bf:isHierarchical  "";')
  for tag in tagsets[ts]:
    if not tag.startswith("Undefined") and tag!="":
     fo.write('\n\t\t\t rdfs:subClassOf [ a owl:Restriction ; owl:onProperty :hasTag ; owl:someValuesFrom tag:'+tag+' ];')
  fo.write('\n\t\t\t rdfs:label "'+ts.replace('_',' ')+'"@en .\n')

In [ ]:
#Write the synonyms
for eqTS in equivTagSets:
  ots=None
  for ts in eqTS:
    if ots:
      fo.write('\n :'+ts+'   owl:equivalentClass  :'+ots+' .')
      #fo.write('\n :'+ts+'   bf:similarClass    :'+ots+' .')
    ots=ts;   

In [ ]:
fo.close()